# Exercise 04 - Due Friday, October 21 at 12pm

*Objectives*: Gain experience loading a CSV dataset into a database model you define yourself and using SQL to explore its contents. Explore the data by writing and executing a number of SQL queries using common syntax and functions and describing your findings.

*Grading criteria*: The tasks should all be completed, and questions should all be answered with SQL queries in the space provided, unless a text answer is requested. Results should be correct, according to the embedded tests. The notebook itself should be completely reproducible; from start to finish, another person should be able to use the same code to obtain the same results as yours.  Note that you will receive no more than partial credit if you do not add text/markdown cells explaining your thinking at each major step in each problem.

*Deadline*: Friday, October 21, 12pm. Submit your notebook to Blackboard and push it to your GitHub repository.

## Setup - obtain data and create database

The US City Open Data Census has data on [restaurant inspections from all over the country](http://us-city.census.okfn.org/dataset/food-safety).  Let's take a look at data from Albuquerque.  *Note*: metadata and other details for this dataset are available at http://data.cabq.gov/business/foodinspections/.

Fyi, to save you a step, I've converted this file into a clean CSV file by using the `dos2unix` command to change its line endings and to switch it to use commas instead of tabs using `csvformat -t inputfile.csv > outputfile.csv`.

First we download the dataset:

In [1]:
!wget https://raw.githubusercontent.com/gwsb-istm-6212-fall-2016/syllabus-and-schedule/master/exercises/abq-food-inspections-fy2010.csv

--2016-10-24 02:07:30--  https://raw.githubusercontent.com/gwsb-istm-6212-fall-2016/syllabus-and-schedule/master/exercises/abq-food-inspections-fy2010.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.32.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.32.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5025226 (4.8M) [text/plain]
Saving to: ‘abq-food-inspections-fy2010.csv’

abq-food-inspection 100%[===================>]   4.79M  12.7MB/s    in 0.4s    

2016-10-24 02:07:31 (12.7 MB/s) - ‘abq-food-inspections-fy2010.csv’ saved [5025226/5025226]



## Problem 1 - examine the data (35 points)

Use `csvstat` and other tools as you see fit and observe its contents.  Review the metadata definitions linked to above.  Describe what you see:  What do all the columns mean?  Are there null values?  Which columns interest you the most?  Which columns present some complications or questions you would like to ask?

Insert a combination of code and text/markdown cells here to review the data and describe what you find.

In [2]:
!mv abq-food-inspections-fy2010.csv 2010.csv

In [3]:
!wc -l 2010.csv

17557 2010.csv


In [4]:
!csvcut -n 2010.csv

  1: FACILITY_NAME
  2: FACILITY_KEY
  3: SITE_ADDRESS
  4: CITY
  5: STATE
  6: ZIP
  7: OWNER_KEY
  8: OWNER_NAME
  9: NATURE_OF_BUSINESS
 10: STREET_NUMBER
 11: STREET_NAME
 12: STREET_TYPE
 13: POST_DIRECTIONAL
 14: PHONE
 15: PROGRAM_CATEGORY
 16: PROGRAM_CATEGORY_DESCRIPTION
 17: INSPECTION_DATE
 18: INSPECTION_TYPE
 19: INSPECTION_DESC
 20: SERIAL_NUM
 21: ACTION_CODE
 22: ACTION_DESC
 23: RESULT_CODE
 24: RESULT_DESC
 25: VIOLATION_CODE
 26: VIOLATION_DESC
 27: INSPECTION_MEMO


In [5]:
!csvstat 2010.csv

  1. FACILITY_NAME
	<class 'str'>
	Nulls: False
	Unique values: 2615
	5 most frequent values:
		SMITHS FOOD AND DRUG CENTERS INC:	154
		ISOTOPES PARK - OVATIONS:	93
		PER EHSII CECELIA GARCIA OOB LA REYNA DEL SUR:	85
		ISLETA AMPHITHEATER:	68
		ECHO INC:	68
	Max length: 77
  2. FACILITY_KEY
	<class 'int'>
	Nulls: False
	Min: 13
	Max: 101482
	Sum: 820405773
	Mean: 46730.79135338346
	Median: 50866.0
	Standard Deviation: 32659.794578837962
	Unique values: 2835
	5 most frequent values:
		46323:	93
		65863:	85
		80949:	68
		32817:	68
		91825:	66
  3. SITE_ADDRESS
	<class 'str'>
	Nulls: True
	Unique values: 2456
	5 most frequent values:
		1601 AVENIDA CESAR CHAVEZ SE:	94
		644 OLD COORS BLVD SW:	85
		8601 CENTRAL AV NE:	77
		2200 SUNPORT BLVD SE:	75
		5210 CENTRAL AV SE:	73
	Max length: 35
  4. CITY
	<class 'str'>
	Nulls: False
	Values: ALBUQUERQUE
  5. STATE
	<class 'str'>
	Nulls: False
	Values: NM, 87
  6. ZIP
	<class 'str'>
	Nulls: False
	Unique values: 48
	5 most frequent values:
		87110

This set of data is records of facility inspections in 2010 in Albuquerque. 

There are total 2835 facilities identified by FACILITY_KEY in the table.

There are 27 colums and 17556 rows of records (exclude row that contains names of colums). Each row is identified by the unique serial number of the inspection(SERIAL_NUM). There are missing values in columns 'NSPECTION_TYPE' and 'INSPECTION_MEMO'.



## Problem 2 - define a database model and load the data (35 points)

Based on what you found above, create and connect to a new database, define a database table in it, and load this dataset into it.  You may use either of the methods for this step you have seen in earlier class notebooks.  You may choose to eliminate variables/columns if they are not relevant or interesting to you - explain your reasoning if you do.  Either way, you should load a majority of the columns present in the source dataset into the database and all of its rows.

Once your data has loaded successfully, run a `COUNT(*)` query to verify that all the data has loaded correctly.

Insert a combination of code and text/markdown cells here to connect to your database, define your table, load its data, and verify the loaded records.

### Eliminating Columns
First I chose to eliminate nominal variables that contain only numbers and the distribution of which cannot be intepreted. These variables are 'OWNER_KEY', 'STREET_NUMBER', 'PROGRAM_CATEGORY', 'SERIAL_NUM','ACTION_CODE', 'RESULT_CODE', and 'VIOLATION_CODE'. 

Columns without value are also deleted. They are 'INSPECTION_TYPE' and 'INSPECTION_MEMO'. 

I also deleted 'CITY' and 'STATE' because the whole column have the same value. 

And 'STREET_TYPE' is irrelevent.

In [6]:
!csvcut -c1,2,3,6,8,9,11,13,14,16,17,19,22,24,26 2010.csv > q1.csv

the remaining columns

In [7]:
!csvcut -n q1.csv

  1: FACILITY_NAME
  2: FACILITY_KEY
  3: SITE_ADDRESS
  4: ZIP
  5: OWNER_NAME
  6: NATURE_OF_BUSINESS
  7: STREET_NAME
  8: POST_DIRECTIONAL
  9: PHONE
 10: PROGRAM_CATEGORY_DESCRIPTION
 11: INSPECTION_DATE
 12: INSPECTION_DESC
 13: ACTION_DESC
 14: RESULT_DESC
 15: VIOLATION_DESC


In [8]:
%load_ext sql

/home/qy/anaconda3/lib/python3.5/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/home/qy/anaconda3/lib/python3.5/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [9]:
!csvsql --db sqlite:///q1.db --insert q1.csv

(sqlite3.OperationalError) table q1 already exists [SQL: '\nCREATE TABLE q1 (\n\t"FACILITY_NAME" VARCHAR(77) NOT NULL, \n\t"FACILITY_KEY" INTEGER NOT NULL, \n\t"SITE_ADDRESS" VARCHAR(35), \n\t"ZIP" VARCHAR(10) NOT NULL, \n\t"OWNER_NAME" VARCHAR(82) NOT NULL, \n\t"NATURE_OF_BUSINESS" VARCHAR(25), \n\t"STREET_NAME" VARCHAR(24), \n\t"POST_DIRECTIONAL" VARCHAR(4), \n\t"PHONE" BIGINT, \n\t"PROGRAM_CATEGORY_DESCRIPTION" VARCHAR(40), \n\t"INSPECTION_DATE" DATE NOT NULL, \n\t"INSPECTION_DESC" VARCHAR(32) NOT NULL, \n\t"ACTION_DESC" VARCHAR(34) NOT NULL, \n\t"RESULT_DESC" VARCHAR(37) NOT NULL, \n\t"VIOLATION_DESC" VARCHAR(50)\n)\n\n']


In [10]:
%sql sqlite:///q1.db

'Connected: None@q1.db'

Verify that all rows from csv file are in the database.

In [11]:
%%sql
SELECT COUNT(*)
FROM q1;

Done.


COUNT(*)
17556


## Problem 3 - explore your data (30 points)

Now that the data is loaded, ask some questions of it!  Identify key variables of interest and note their ranges along with other useful descriptive statistics.  Choose and define a few lines of inquiry, execute queries for each, and describe what you find in your result sets.  Use any query techniques we've seen in class, including aggregate functions, transformations, subqueries, or others as appropriate. 

If you find interesting patterns, adding plots to your exploration would be useful.

Insert a combination of code and text/markdown cells here to explore your data.

### Create a new table to house the unique facilities and thier information.

In [13]:
%%sql
DROP TABLE IF EXISTS Facilities;
CREATE TABLE Facilities
(
    FACILITY_KEY       INTERGER PRIMARY KEY,
    FACILITY_NAME      VARCHAR(64),
    NATURE_OF_BUSINESS VARCHAR(64),
    OWNER_NAME         VARCHAR(64), 
    STREET_NAME        VARCHAR(64)
);

Done.
Done.


[]

In [14]:
%%sql
INSERT INTO Facilities
SELECT DISTINCT FACILITY_KEY, FACILITY_NAME, NATURE_OF_BUSINESS, OWNER_NAME, STREET_NAME 
FROM q1 ORDER BY FACILITY_KEY ASC;

2835 rows affected.


[]

Verify that all facilities from csv file are in the database.

In [15]:
%%sql
SELECT COUNT(*)
FROM Facilities;

Done.


COUNT(*)
2835


In [16]:
%%sql
SELECT *
FROM Facilities
LIMIT 10;

Done.


FACILITY_KEY,FACILITY_NAME,NATURE_OF_BUSINESS,OWNER_NAME,STREET_NAME
13,BELIEVERS CENTER ALBUQUERQUE,COFFEE BAR,BCA COFFEE BAR,WATERFALL
16,LITTLE CORRAL DAY SCHOOL,CHILD CARE,LITTLE CORRAL DAY SCHOOL,CONSTITUTION
29,POWDRELLS BAR B Q,RESTAURANT,DRELLS INC,4TH
144,OOB 2/26/14PEPSI COLA BOTTLING,WHOLESALER,OOB 2/26/14PEPSI COLA METR BOTLG,CLAREMONT
148,OOB EUBANK DAY SCHOOL/CHILDCARE,CHILD CARE,EUBANK DAY SCHOOL/CHILDCARE,EUBANK
150,ROBERTS OIL,SERVICE STATION,ROBERTS OIL,CENTRAL
152,ROBERTS OIL 4,SERVICE STATION,ROBERTS OIL,COORS
153,PHILLIPS 66 / ROBERTS OIL,SERVICE STATION,ROBERTS OIL,MENAUL
154,ROBERTS OIL CO 18,SERVICE STATION,ROBERTS OIL,MENAUL
155,ROBERTS OIL CO,SERVICE STATION,ROBERTS OIL,WYOMING


### Create another database to house the rate of approval of each facility
I only keep facilities that HAVE approved actions

In [17]:
%%sql
SELECT COUNT(DISTINCT FACILITY_KEY)
FROM q1
WHERE ACTION_DESC = 'APPROVED'

Done.


COUNT(DISTINCT FACILITY_KEY)
2710


In [18]:
%%sql
DROP TABLE IF EXISTS RESULT;
CREATE TABLE RESULT
(
    FACILITY_KEY       INTERGER PRIMARY KEY,
    FACILITY_NAME      VARCHAR(64),
    NUM_APPROVED       INTEGER,
    TOTAL_INSPECTION   INTEGER,
    APPROVAL_RATE      INTEGER
);

Done.
Done.


[]

In [19]:
%%sql
INSERT INTO RESULT(FACILITY_KEY, FACILITY_NAME)
SELECT DISTINCT FACILITY_KEY, FACILITY_NAME FROM q1 WHERE ACTION_DESC = 'APPROVED';

2710 rows affected.


[]

In [20]:
%%sql
UPDATE RESULT
SET NUM_APPROVED = (SELECT COUNT(*) FROM q1
                    WHERE ACTION_DESC = 'APPROVED' AND RESULT.FACILITY_KEY = q1.FACILITY_KEY);

2710 rows affected.


[]

In [21]:
%%sql
UPDATE RESULT
SET TOTAL_INSPECTION = (SELECT COUNT(*) FROM q1
                    WHERE RESULT.FACILITY_KEY = q1.FACILITY_KEY);

2710 rows affected.


[]

In [22]:
%%sql
UPDATE RESULT
SET APPROVAL_RATE = (NUM_APPROVED*10)/(TOTAL_INSPECTION);

2710 rows affected.


[]

Verify data of all facilities with approved actions is included

In [23]:
%%sql
SELECT COUNT(*)
FROM RESULT;

Done.


COUNT(*)
2710


### Rate of Approval
Below is the distribution of approval rate of the facilities. We can see that there are 29%(790/2710) of the facilities got approval in every inspections. 
e.g. score = 7 means the rate of approval is between 60% and 69%

In [24]:
%%sql
SELECT APPROVAL_RATE AS SCORE, COUNT(*) AS COUNT
FROM RESULT
GROUP BY APPROVAL_RATE
ORDER BY APPROVAL_RATE DESC
LIMIT 20;

Done.


SCORE,COUNT
10,790
9,143
8,1209
7,183
6,225
5,92
4,20
3,17
2,11
1,15


### Top 10 nature of business in Albuquerque

In [25]:
%%sql
SELECT NATURE_OF_BUSINESS, COUNT(*) AS COUNT
FROM Facilities
WHERE NATURE_OF_BUSINESS != 'None'
GROUP BY NATURE_OF_BUSINESS
ORDER BY COUNT DESC
LIMIT 10;

Done.


NATURE_OF_BUSINESS,COUNT
RESTAURANT,834
FOOD SERVICE,133
GROCERY,104
APS SCHOOL FOOD SERVICE,62
BAR/RESTAURANT,51
INST KITCHEN,50
APS SCHOOL FOODSERVICE,48
CHILD CARE,45
HOTEL,38
GAS STATION,36


### Top 10 streets with most restaurants

In [26]:
%%sql
SELECT STREET_NAME, COUNT(*) AS COUNT
FROM Facilities
WHERE NATURE_OF_BUSINESS LIKE '%RESTAURANT%'
GROUP BY STREET_NAME
ORDER BY COUNT DESC
LIMIT 10;

Done.


STREET_NAME,COUNT
CENTRAL,138
COORS,67
MENAUL,63
SAN MATEO,61
WYOMING,52
MONTGOMERY,47
EUBANK,42
JUAN TABO,35
4TH,33
LOMAS,25


### Inspection Results
Below is the distribution of routine food inspection results. We can see that 12245 of total 12439 results (over 98%) are in compliance.

In [28]:
%%sql
SELECT RESULT_DESC, COUNT(*) AS COUNT
FROM q1
WHERE INSPECTION_DESC = 'ROUTINE FOOD INSPECTION'
GROUP BY RESULT_DESC
ORDER BY COUNT DESC;

Done.


RESULT_DESC,COUNT
IN COMPLIANCE,12245
DOWNGRADE,156
NOT APPLICABLE,21
IMMEDIATE CLOSURE,9
NOTICE,3
NOT IN COMPLIANCE,2
PERMIT SUSPENSION,2
VOLUNTARY CLOSURE,1


In [29]:
%%sql
SELECT COUNT(*)
FROM q1
WHERE INSPECTION_DESC = 'ROUTINE FOOD INSPECTION';

Done.


COUNT(*)
12439


## Bonus (10 points)

We have seen a few simple techniques for normalizing data in class.  Choose a column or set of columns that might be redundant and normalize them out to another table:

* Identify the columns to normalize in the source data table
* Create one or more new tables to hold the normalized records
* Extract data into the new tables
* Re-insert the new foreign key references from the new table into the source table

In my understanding, the two databases I created above are kind of normalized. 